# Imports

In [57]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
# from pennylane.templates import AmplitudeEmbedding
from pennylane.templates import AngleEmbedding

# Utils

In [58]:
class_to_prediction = {
    'Setosa': 0.33333,
    'Versicolor': 0.66667,
    'Virginica': 1.00000,
}

# Model Params

In [59]:
np.random.seed(42)
initial_params = np.random.random([7])

variety_to_ignore = 'Setosa'

steps = 80*100
step_size = 0.3

# Import training dataset

In [60]:
columns = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width']
dataset = pd.read_csv('iris_train.csv', usecols=columns)
dataset_class = pd.read_csv('iris_train.csv', usecols=['variety', 'variety.class'])

In [61]:
dataset = dataset.drop(dataset[dataset_class.variety == variety_to_ignore].index)
print(dataset)
dataset = dataset.values.tolist()

     sepal.length  sepal.width  petal.length  petal.width
40            7.0          3.2           4.7          1.4
41            6.4          3.2           4.5          1.5
42            6.9          3.1           4.9          1.5
43            5.5          2.3           4.0          1.3
44            6.5          2.8           4.6          1.5
..            ...          ...           ...          ...
115           7.7          3.0           6.1          2.3
116           6.3          3.4           5.6          2.4
117           6.4          3.1           5.5          1.8
118           6.0          3.0           4.8          1.8
119           6.9          3.1           5.4          2.1

[80 rows x 4 columns]


In [62]:
dataset_class['variety.class'] = dataset_class['variety.class'] / dataset_class['variety.class'].max()
dataset_class = dataset_class.drop(dataset_class[dataset_class.variety == variety_to_ignore].index)
print(dataset_class)
dataset_class = dataset_class.values.tolist()

        variety  variety.class
40   Versicolor       0.666667
41   Versicolor       0.666667
42   Versicolor       0.666667
43   Versicolor       0.666667
44   Versicolor       0.666667
..          ...            ...
115   Virginica       1.000000
116   Virginica       1.000000
117   Virginica       1.000000
118   Virginica       1.000000
119   Virginica       1.000000

[80 rows x 2 columns]


# Circuit creation

In [63]:
device = qml.device("default.qubit", wires=4)

In [64]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    # AmplitudeEmbedding(features=features, wires=range(4), normalize=True, pad_with=0.)
    AngleEmbedding(features=features, wires=range(4), rotation='Y')

    # First layer
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[3, 2])

    # Second layer
    qml.RY(params[4], wires=1)
    qml.RY(params[5], wires=2)
    qml.CNOT(wires=[1, 2])

    # Third layer
    qml.RY(params[6], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

In [65]:
features = dataset[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361]

Example features: [7.0, 3.2, 4.7, 1.4]

Expectation value: 0.06261129521637804

 0: ──RY(7)────RY(0.375)──╭C─────────────────────────────┤     
 1: ──RY(3.2)──RY(0.951)──╰X──RY(0.156)──╭C──────────────┤     
 2: ──RY(4.7)──RY(0.732)──╭X──RY(0.156)──╰X──RY(0.0581)──┤ ⟨Z⟩ 
 3: ──RY(1.4)──RY(0.599)──╰C─────────────────────────────┤     



# Training

In [66]:
params = initial_params
opt = qml.GradientDescentOptimizer(stepsize=step_size)

for i in range(steps):
    features = dataset[i%len(dataset)]
    expected_value = dataset_class[i%len(dataset)][1]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(dataset)

    params = opt.step(cost, params)

In [67]:
print("Optimized rotation angles: {}".format(params))

Optimized rotation angles: [-0.92880227  0.8036941   0.02965781  0.89312597 -0.82361614  0.28385674
  0.94465008]


# Import testing dataset

In [68]:
dataset_test = pd.read_csv('iris_test.csv', usecols=columns)
dataset_class_test = pd.read_csv('iris_test.csv', usecols=['variety', 'variety.class'])

In [69]:
dataset_test = dataset_test.drop(dataset_test[dataset_class_test.variety == variety_to_ignore].index)
print(dataset_test)
dataset_test = dataset_test.values.tolist()

    sepal.length  sepal.width  petal.length  petal.width
10           5.5          2.6           4.4          1.2
11           6.1          3.0           4.6          1.4
12           5.8          2.6           4.0          1.2
13           5.0          2.3           3.3          1.0
14           5.6          2.7           4.2          1.3
15           5.7          3.0           4.2          1.2
16           5.7          2.9           4.2          1.3
17           6.2          2.9           4.3          1.3
18           5.1          2.5           3.0          1.1
19           5.7          2.8           4.1          1.3
20           6.7          3.1           5.6          2.4
21           6.9          3.1           5.1          2.3
22           5.8          2.7           5.1          1.9
23           6.8          3.2           5.9          2.3
24           6.7          3.3           5.7          2.5
25           6.7          3.0           5.2          2.3
26           6.3          2.5  

In [70]:
dataset_class_test['variety.class'] = dataset_class_test['variety.class'] / dataset_class_test['variety.class'].max()
dataset_class_test = dataset_class_test.drop(dataset_class_test[dataset_class_test.variety == variety_to_ignore].index)
print(dataset_class_test)
dataset_class_test = dataset_class_test.values.tolist()

       variety  variety.class
10  Versicolor       0.666667
11  Versicolor       0.666667
12  Versicolor       0.666667
13  Versicolor       0.666667
14  Versicolor       0.666667
15  Versicolor       0.666667
16  Versicolor       0.666667
17  Versicolor       0.666667
18  Versicolor       0.666667
19  Versicolor       0.666667
20   Virginica       1.000000
21   Virginica       1.000000
22   Virginica       1.000000
23   Virginica       1.000000
24   Virginica       1.000000
25   Virginica       1.000000
26   Virginica       1.000000
27   Virginica       1.000000
28   Virginica       1.000000
29   Virginica       1.000000


# Testing

In [71]:
del class_to_prediction[variety_to_ignore]

for index in range(len(dataset_test)):
    expected_class = dataset_class_test[index][0]
    expected_value = dataset_class_test[index][1]

    features = dataset_test[index]
    value = circuit(features, params)

    predicted_class = ''
    current_diff = 10000000000000000000
    for iris_class, prediction_value in class_to_prediction.items():
        class_diff = np.sqrt((prediction_value - value) ** 2)
        if class_diff > current_diff:
            continue

        predicted_class = iris_class
        current_diff = class_diff

    if predicted_class != expected_class:
        print(f'Error in prediction. Expected class: {expected_class}. Predicted_class: {predicted_class}')
        print(f'MSE: {current_diff}')

Error in prediction. Expected class: Virginica. Predicted_class: Versicolor
MSE: 0.1540474633159511
Error in prediction. Expected class: Virginica. Predicted_class: Versicolor
MSE: 0.14369023084627075
Error in prediction. Expected class: Virginica. Predicted_class: Versicolor
MSE: 0.159915621218225
